## Asignacion 2

Todos los asignaciones serán presentadas utilizando los cuadernos de [Jupyter Notebook](http://jupyter.org/), además de respectivas pruebas como ejemplo, así como el uso de mediciones de velocidad de ejecución utilizando el comando `timeit` y algunos gráficos si fuese necesario.

### Preguntas

1. Presenta un reporte del siguiente artículo [Data Science, Machine Learning, and Statistics: what is in a name?](http://www.win-vector.com/blog/2013/04/data-science-machine-learning-and-statistics-what-is-in-a-name/).

2. Revisa el simpático proyecto kickstarter [The Emoji Translation Project :The world needs an emoji translation engine. Help us build it using the power of crowdsourcing. ](https://www.kickstarter.com/projects/fred/the-emoji-translation-project). 

3. Leer el artículo de Paul Graham [A Plan for Spam](http://www.paulgraham.com/spam.html) y responder las siguientes  preguntas de acuerdo a lo que dice el autor y lo que piensas tu:

   - ¿Cuáles son los componentes clave de su sistema de filtrado estadístico? En otras palabras, ¿cómo funciona?.
   - Presenta una lista de beneficios del enfoque estadístico.

4. Presenta un reporte de tus ideas con ejemplos  del siguiente artículo [Choosing a Machine Learning Classifier](http://blog.echen.me/2011/04/27/choosing-a-machine-learning-classifier/).

## Lista de ejercicios

Las referencias a los ejercicios son :

   * Asignaciones de Andrew Ng de Stanford University y cursos en Coursera.
   * David Barber, [Bayesian Reasoning and Machine Learning](http://web4.cs.ucl.ac.uk/staff/D.Barber/pmwiki/pmwiki.php?n=Brml.Online), Cambridge University Press, 2017.
   * Pattern Recognition and Machine Learning de Chris Bishop 2006.
   * Machine Learning Refined, Watt, Borhani, Katsaggelos 2016.


1 . Este conjunto de problemas implica la implementación de varias variantes del algoritmo Perceptron. Antes de poder construir estos modelos y medir su rendimiento, divide tus datos de entrenamiento (es decir, `entrenamiento_spam.txt`) en un conjunto de entrenamiento y validación, poniendo los últimos 1000 correos electrónicos en el conjunto de validación. Por lo tanto, tendrá un nuevo conjunto de entrenamiento con 4000 correos electrónicos y un conjunto de validación con 1000 correos electrónicos. 

Explica por qué medir el rendimiento de su clasificador final sería problemático si no hubiera creado este conjunto de validación.

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('entrenamiento_spam.txt')#,sep="\s",header=None)

In [3]:
data.columns = ['Text']
display(data.head())

# Separamos la columna de Spam
df = pd.DataFrame(data.Text.str.split(' ',1).tolist(),columns=['Spam','Text'])
display(df.head())

,Text
0,1 have tax problem do you ow the ir monei if y...
1,0 r robert harlei write r scuse me for post in...
2,0 on wed number oct number brian fahrland wrot...
3,0 quot ronan waid sure but soft link would do ...
4,0 i notic a low count of razor d spam messag s...


,Spam,Text
0,1,have tax problem do you ow the ir monei if you...
1,0,r robert harlei write r scuse me for post in g...
2,0,on wed number oct number brian fahrland wrote ...
3,0,quot ronan waid sure but soft link would do th...
4,0,i notic a low count of razor d spam messag so ...


In [10]:
# Separamos los correos de entrenamiento y validación
#X_train = df[:4000].Word
#X_test = df[4000:].Word
X = df['Text']
y = df['Spam']
test_size = 1.0/5.0
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size,random_state=2)
print (len(X_train))
X_train.head()

3999


1305    todai s headlin from the regist to unsubscrib ...
2525    here s clarif of why i did first test result u...
3966    newslett coachnvest septembr number coachnvest...
3394    onc upon a time vill wrote i assum that you wo...
2961    ar your tire of number to number let us show y...
Name: Text, dtype: object

2 . Transforme todos los datos en vectores de características. Construya una lista de vocabulario usando solamente el conjunto de entrenamiento de 4000 correos electrónicos encontrando todas las palabras que aparecen en el conjunto de entrenamiento.

Ten en cuenta que suponemos que los datos de los conjuntos de validación y prueba no se ven completamente cuando entrenamos nuestro modelo y, por lo tanto, no utilizamos ninguna información contenida en ellos. 

Ignora todas las palabras que aparecen en menos de X = 30 correos electrónicos del conjunto de 4000 mensajes de correo electrónico, esto es tanto un medio de prevenir el ajuste excesivo y de mejorar la escalabilidad. Para cada correo electrónico, transformarlo en un vector de características $\mathbf{x}$ donde la i-ésima entrada, $x_i$, es 1 si la i-ésima palabra del vocabulario aparece en el correo electrónico y 0 en caso contrario.

In [11]:
#Separamos las palabras y calculamos su frecuencia para todos los correos
from collections import Counter
word_freq = Counter(" ".join(X_train.str.lower().values.tolist()).split(" ")).items()

# Colocamos las frecuencias en tablas y restringimos para frecuencias mayores a 30
wf = pd.DataFrame(word_freq,columns=['Word','Freq'])
wf = wf[wf['Freq']>=30]
print (len(wf))
wf.head()

3251


,Word,Freq
45,showcas,55
56,wednesdai,97
94,china,122
136,music,191
147,yahoo,311


In [24]:
# Generamos las columnas adicionales
print (len(X_train))
for word in wf.Word:
    serie = []
    for i in range(0,len(X_train)):
        if word in X_train.iloc[i]:
            serie.append(1)
        else:
            serie.append(0)
    X_train[word]=serie
X_train['china']
    #if word in X_train.iloc[i]['Text']:
        #    X_train.iloc[i][word] = 1


7250


KeyboardInterrupt: 

3 .Implementar las funciones **entrenamiento_perceptron(datos)** y **prueba_perceptron(w, datos)**.

La función `entrenamiento_perceptron(datos)` entrena un clasificador perceptron usando los ejemplos proporcionados a la función, y devuelve `w`, `k`, `iter`, el vector de clasificación final, el número de actualizaciones (errores) realizados y el número de pasadas los datos, respectivamente. 

Se puede  suponer que los datos de entrada proporcionados a la función son linealmente separables (por lo que el criterio de parada debe ser que todos los puntos estén clasificados correctamente).

Para el caso  `w·x = 0`, predecir la clase +1 (spam). Para este ejercicio, no es necesario agregar una función de sesgo al vector de características 

La función `prueba_perceptron(w, datos)` debe tomar como entrada el vector de peso `w`  (el vector de clasificación a utilizar) y un conjunto de ejemplos. La función debe devolver el error de prueba, es decir, la fracción de ejemplos que son mal clasificados por w.

In [ ]:
# Tu respuesta

4 .Entrene el clasificador lineal usando su conjunto de entrenamiento. ¿Cuántos errores se cometen antes de que termine el algoritmo? Prueba tu implementación `prueba_perceptron` con  los parámetros aprendidos y los datos de entrenamiento, asegurándose de que el error de entrenamiento es cero. A continuación, clasifique los correos electrónicos en su conjunto de validación. ¿Qué es el error de validación?

In [ ]:
# Tu respuesta

5 .Para entender mejor cómo funciona este clasificador de spam, podemos inspeccionar los parámetros para ver qué palabras cree que el clasificador es la más predictiva a spam. Usando la lista de vocabulario junto con los parámetros aprendidos en la pregunta anterior, muestra las 15 palabras con los pesos más positivos. ¿Cuáles son las  15 palabras que tienen los pesos más negativos?.


In [ ]:
# Tu respuesta

6 . Implementar el algoritmo de perceptron promedio, que es el mismo que tu implementación actual, pero que, en lugar de devolver el vector de peso final, devuelve el promedio de todos los vectores de peso considerados durante el algoritmo (incluyendo ejemplos donde no se cometió ningún error). El promedio reduce la varianza entre los diferentes vectores, y es un poderoso medio para evitar que el algoritmo de aprendizaje sea sobrefijado (sirviendo como un tipo de regularización).

In [ ]:
# Tu respuesta

7 .Uno debe esperar que el error de prueba disminuye a medida que aumenta la cantidad de datos de entrenamiento. Usando sólo las primeras N filas de tus datos de entrenamiento, ejecute tanto el algoritmo del perceptron como el  algoritmo de perceptron promedio en este conjunto de entrenamiento  y evalúe el error de validación correspondiente (usando todos los datos de validación). Hacer esto para N = 100, 200, 400, 800, 2000, 4000 y crear un gráfico del error de validación de ambos algoritmos como una función de N.

In [ ]:
# Tu respuesta

8 . También para N = 100, 200, 400, 800, 2000, 4000, crea un diagrama del número de iteraciones para el perceptrón como una función de N, donde por iteración nos referimos a un paso completo a través de los datos de entrenamiento. A medida que aumenta la cantidad de datos de entrenamiento, el margen del conjunto de entrenamiento disminuye, lo que generalmente conduce a un aumento en el número de iteraciones que el perceptron toma para converger.


In [ ]:
# Tu respuesta

9 . Prueba varias configuraciones de los algoritmos usando todos los 4000 datos de entrenamiento y encuentra una buena configuración con un error pequeño en su conjunto de validación. En particular, trata de cambiar la elección del algoritmo perceptron y el número máximo de iteraciones. 

Reporta el error de validación para varias de las configuraciones, qué configuración funciona mejor?

Combina el conjunto de entrenamiento y el conjunto de validación (es decir, utiliza `entrenamiento_spam.txt`) y aprenda usando la mejor de las configuraciones encontradas anteriormente.

¿Cuál es el error en el conjunto de pruebas, es decir, en  `prueba_spam.txt`?

In [ ]:
# Tu respuesta